# Unigram & Bigram Model

In [13]:
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Unigram
To flag words the model has never seen before.

In [34]:
'''
Class Unigram: for unigram language model.

Attributes:
- n_unigrams: number of tokens in training data
- vocab_size: number of unique unigram in training data
- count: a dictionary of unigram counts

Methods:
- train(train_data): train the unigram model
- compute_prob(unigram): compute the probability of a unigram using
                        unigram count and add-one smoothing
- test_perplexity(test_data): compute the perplexity of test data using
                        log likelihood method to avoid underflow
'''

class Unigram:
    def __init__(self):
        self.n_unigram = 0
        self.vocab_size = 0
        self.count = {}

    def train(self, train_data):
        '''
        This function trains the unigram model. After running this funtion,
        a dictionary storing counts of each unigram will be created.

        Input:
            train_data: string
                training data to train the unigram model
        Return:
            None
        '''
        # tokenize & assigning values to model attributes
        unigrams = nltk.tokenize.word_tokenize(train_data)
        # self.count["<UNK>"] = 0 -- REMOVE UNK TOKEN FOR FLAGGING TYPO
        self.n_unigram = len(unigrams)
        self.vocab_size = len(set(unigrams))

        # creating unigram count dictionary
        for unigram in unigrams:
            if unigram in self.count.keys():
                self.count[unigram] += 1
            else:
                self.count[unigram] = 1


    def compute_prob(self, unigram):
        '''
        This function takes in a unigram, and returns the probability that
        unigram appears in the training data with add-one smoothing.

        Input:
            unigram: string
                the unigram to compute probability for
        Return:
            the probability of the given unigram appearing in the training data
        '''
        N = self.n_unigram
        V = self.vocab_size
        if (unigram in self.count.keys()):
            return (self.count[unigram] + 1) / (N + V) # smoothing
        else:
            return 1 / (N + V) # smoothing for unseen words

    def test_perplexity(self, test_data):
        '''
        This function takes in a test data, and returns the perplexity
        of this data on the unigram model. The perplexity is computed
        using log likelihood method to avoid underflow.

        Input:
            test_data: string
                the test data to compute perplexity for
        Return:
            the perplexity of the test data on the unigram model
        '''
        test_unigrams = nltk.tokenize.word_tokenize(test_data)
        M = len(test_unigrams)

        # perplexity = exponential of negative average log likelihood
        probs = []
        for unigram in test_unigrams:
            probs.append(self.compute_prob(unigram))

        avg_log_likelihood = np.log(probs).sum() / M
        ppl = np.exp((-1) * avg_log_likelihood)
        return ppl

    def flag_unseen(self, test_data):
        '''
        This function takes in a sentence, and returns the lists of words
        that might be a typo. A word is considered a typo if it is not in
        the training data.

        Input:
            test_data: string
                the test data to flag unseen words for
        Return:
            unseen: list of strings
                the list of words that might be a typo
        '''
        test_unigrams = nltk.tokenize.word_tokenize(test_data)
        unseen = []
        for unigram in test_unigrams:
            if unigram not in self.count.keys():
                unseen.append(unigram)
        return unseen

## Bigram
To flag pairs of words that seems to not often go together.

In [35]:
'''
Class Bigram: for bigram language model.

Attributes:
- count_unigram: a dictionary of unigram counts
- count_bigram: a dictionary of bigram counts
- n_unigrams: number of unigram in training data
- n_bigrams: number of bigram tokens in training data
- vocab_size: number of unique unigram in training data


Methods:
- get_ngram(n, text): get ngrams from text
- count_ngram(ngrams): count ngrams frequency
- train(train_data): train the bigram model
- compute_prob(bigram): compute the probability of a bigram appearing
                        using add-one smoothing
- test_perplexity(test_data): compute the perplexity of test data using
                        log likelihood method to avoid underflow
'''

class Bigram:
    def __init__(self):
        self.count_unigram = {}
        self.count_bigram = {}
        self.n_unigram = 0
        self.n_bigram = 0

        self.vocab_size = 0


    def get_ngram(self, n, text):
        '''
        This function takes in a text and returns a list of ngrams.
        If n = 1, return a list of unigrams using nltk.tokenize module.
        Higher ngrams are manually created with sliding window.

        Input:
            n: int
                the n in 'ngram'
            text: string
                the text to be split into ngrams

        Return:
            a list of ngrams
        '''
        unigrams = nltk.tokenize.word_tokenize(text)
        ngrams = []
        if n == 1:
            return unigrams
        else:
            last_start = len(unigrams) - n + 1
            for i in range(last_start):
                ngram = tuple(unigrams[i: i+n])
                ngrams.append(ngram)
            return ngrams

    def count_ngram(self, ngrams):
        '''
        This function takes in a list of ngrams and returns a dictionary
        counting the frequency of each ngram.

        Input:
            ngrams: list
                a list of ngrams

        Return:
            a dictionary of ngram counts
        '''
        count = {}
        # count["<UNK>"] = 0

        for ngram in ngrams:
            if ngram in count.keys():
                count[ngram] += 1
            else:
                count[ngram] = 1
        return count

    def train(self, train_data):
        '''
        This function trains the bigram model. After running this funtion,
        two dictionaries storing counts of each unigram and bigram will be
        created.

        Input:
            train_data: string
                training data to train the bigram model
        Return:
            None
        '''
        # tokenize unigrams & assigning unigram attributes
        unigrams = self.get_ngram(1, train_data)
        self.n_unigram = len(unigrams)
        self.vocab_size = len(set(unigrams))
        self.count_unigram = self.count_ngram(unigrams)

        # tokenize bigrams & assigning bigram attributes
        bigrams = self.get_ngram(2, train_data)
        self.n_bigram = len(bigrams)
        self.count_bigram = self.count_ngram(bigrams)

    def compute_prob(self, bigram):
        '''
        This function takes in a bigram, and returns the probability that
        bigram appears in the training data with add-one smoothing.

        In case of unseen bigrams (a, b):
        (1) a is unseen, b is seen:
            P(a, b) = (count(a, b) +  1 )/ (count(a) + V)
                    = 1 / V
        (2) a is seen, b is unseen:
            P(a, b) = count(a, b) + 1 / count(a) + V
                    = 1 / count(a) + V
        (3) a is unseen, b is unseen:
            P(a, b) = count(a, b) + 1 / count(a) + V
                    = 1 / V
        (4) a is seen, b is seen, but in the wrong order:
            P(a, b) = count(a, b) + 1 / count(a) + V
                    = 1 / count(a) + V

        Input:
            bigram: tuple
                the bigram to compute probability for
        Return:
            the probability of the given bigram appearing in the training data
        '''
        ctx = bigram[0]
        if (ctx in self.count_unigram.keys()):
            context = self.count_unigram[ctx]
        else:
            context = 0

        if (bigram in self.count_bigram.keys()):
            joint = self.count_bigram[bigram]
        else:
            joint = 0

        return (joint + 1) / (context + self.vocab_size) # smoothing

    def test_perplexity(self, test_data):
        '''
        This function takes in a test data, and returns the perplexity
        of this data on the bigram model. The perplexity is computed
        using log likelihood method to avoid underflow.

        Input:
            test_data: string
                the test data to compute perplexity for
        Return:
            the perplexity of the test data on the bigram model
        '''
        test_bigrams = self.get_ngram(2, test_data)
        test_unigrams = self.get_ngram(1, test_data)
        M = len(test_unigrams)

        probs = []

        # compute the probability that the first word in test data appears
        # P(first) is the probaility that this unigram appears in training data
        first_word = test_unigrams[0]
        p_first_word = 1
        if (first_word in self.count_unigram.keys()):
            # if first_word is seen
            p_first_word = (self.count_unigram[first_word] + 1) \
                  / (self.n_unigram + self.vocab_size)
        else:
            # first_word is unseen
            p_first_word = 1 / (self.n_unigram + self.vocab_size)

        probs.append(p_first_word)

        for bigram in test_bigrams:
            probs.append(self.compute_prob(bigram))

        avg_log_likelihood = np.log(probs).sum() / M
        ppl = np.exp((-1) * avg_log_likelihood)
        return ppl

    def flag_typo(self, test_data, delta=0.00001):
        '''
        This function takes in a sentence, and returns the list of words
        that might be a typo (spelling mistake) in the sentence.

        A pair of word (bigram) is considered a typo if it is
        has a probability of occurring less than delta in the
        training data.

        *Note: the default delta is arbitrary and can be adjusted.

        Input:
            test_data: string
                the test data to flag typo for
        Return:
            typo: list of strings
                the list of words that might be a typo
        '''
        test_bigrams = self.get_ngram(2, test_data)
        typo = []
        for bigram in test_bigrams:
            prob = self.compute_prob(bigram)
            if prob < delta:
                typo.append(bigram[1])
        return typo

# Spelling Mistake Checking

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [21]:
import json
from datasets import load_dataset

wiki = load_dataset("viettelai/wiki-dump-cleaned", split="train")

In [22]:
# write wiki["text"] to a single txt file
i = 0
SHRINK_FACTOR = 100

with open("small_wiki.txt", "w") as f:
    for text in wiki["text"]:
        if (i%SHRINK_FACTOR==0):
            f.write(text + "\n")
        i += 1

In [23]:
with open("small_wiki.txt", "r") as f:
    data = f.read()

In [36]:
unigram = Unigram()
unigram.train(data)

In [37]:
bigram = Bigram()
bigram.train(data)

In [41]:
def find_spelling_mistakes(text, pretrained_unigram, pretrained_bigram, delta=0.000001):
    print("Câu gốc: ", text)
    print("Những từ có thể bị sai chính tả:")
    print(pretrained_unigram.flag_unseen(text))

    print("Những từ lạ:")
    print(pretrained_bigram.flag_typo(text, delta))

    print("Perplexity của câu: ", pretrained_bigram.test_perplexity(text))

In [29]:
cau_oke = [
    "Trời đang mưa, nhưng tôi vẫn cảm thấy vui vẻ.",
    "Con đường dài, nhưng đích đến luôn gần gũi.",
    "Ngày nay, tôi đã học được nhiều điều mới.",
    "Sự sáng tạo không bao giờ ngừng lại.",
    "Cuộc sống không phải lúc nào cũng dễ dàng, nhưng chúng ta vẫn cố gắng.",
    "Tôi muốn đi du lịch, nhưng tôi không có thời gian.",
]

In [30]:
cau_loi = [
    "Tại sao tao không thấy mày đứng diowis sân trường?",
    "Một điều tôi thích ở Thái Lan là thực ẩm đường phố của họ.",
    "Tôi thích ở nhà vì tôi có thể nấu ăn và việc làm nhà.",
    "Chai nước này đang thoại điện thoại của tôi.",
    "Họ gọi em là thg điên rồi cúp máy luôn.",
    "Ăn cơm với cái gì cx đc, ko ăn cx ksao hết."
]

In [42]:
for cau in cau_oke:
    find_spelling_mistakes(cau, unigram, bigram)


Câu gốc:  Trời đang mưa, nhưng tôi vẫn cảm thấy vui vẻ.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  9656.596333540645
Câu gốc:  Con đường dài, nhưng đích đến luôn gần gũi.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  9776.668923410354
Câu gốc:  Ngày nay, tôi đã học được nhiều điều mới.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  2205.8036100212967
Câu gốc:  Sự sáng tạo không bao giờ ngừng lại.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  4433.043928903044
Câu gốc:  Cuộc sống không phải lúc nào cũng dễ dàng, nhưng chúng ta vẫn cố gắng.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  2938.2384628019713
Câu gốc:  Tôi muốn đi du lịch, nhưng tôi không có thời gian.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  2041.3760648964585


In [43]:
for cau in cau_loi:
    find_spelling_mistakes(cau, unigram, bigram)


Câu gốc:  Tại sao tao không thấy mày đứng diowis sân trường?
Những từ có thể bị sai chính tả:
['diowis']
Những từ lạ:
[]
Perplexity của câu:  39834.49600825416
Câu gốc:  Một điều tôi thích ở Thái Lan là thực ẩm đường phố của họ.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  4577.326628882519
Câu gốc:  Tôi thích ở nhà vì tôi có thể nấu ăn và việc làm nhà.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  5995.664338959581
Câu gốc:  Chai nước này đang thoại điện thoại của tôi.
Những từ có thể bị sai chính tả:
[]
Những từ lạ:
[]
Perplexity của câu:  10164.901389644388
Câu gốc:  Họ gọi em là thg điên rồi cúp máy luôn.
Những từ có thể bị sai chính tả:
['thg']
Những từ lạ:
[]
Perplexity của câu:  39147.75991511004
Câu gốc:  Ăn cơm với cái gì cx đc, ko ăn cx ksao hết.
Những từ có thể bị sai chính tả:
['cx', 'đc', 'cx', 'ksao']
Những từ lạ:
[]
Perplexity của câu:  45757.33185487031
